<a href="https://colab.research.google.com/github/achintya-7/Brain-Tumor-Segmentation/blob/main/Brain_Tumor_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNET

The architecture for UNET is as following

![U-Net Architecture](https://raw.githubusercontent.com/achintya-7/Brain-Tumor-Segmentation/main/model/media/img/u-net-architecture.png)

Importing necessary packages from tensorflow

In [5]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

Defining all the layers of unet as per the diagram

## Conv_block
1. Conv2D: This is a 2D convolution layer, which is a fundamental component of convolutional neural networks (CNNs). It applies a number of convolution operations on the input. The number of these operations is defined by the number of filters. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The transformation is applied on the whole image.

2. BatchNormalization: This is a technique to improve the performance and stability of neural networks by normalizing the inputs in every layer. It reduces the amount the hidden unit values shift around (covariate shift). In other words, it helps each layer of a network to learn by itself a little more independently of other layers.

3. Activation: An activation function defines the output of a neuron given an input or set of inputs. In this case, 'relu' (Rectified Linear Unit) is used as the activation function. The ReLU function is f(x) = max(0, x), where x is the input. It sets all negative values in the matrix x to 0 and keeps all the other values the same.

The conv_block function applies two sets of these layers (Conv2D -> BatchNormalization -> Activation) sequentially. This is a common architecture for a block in a convolutional neural network. The function takes as input a tensor (multi-dimensional array) and the number of filters for the Conv2D layers, and returns the output tensor after applying these operations.

In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

## Encoder Block

1. x = conv_block(inputs, num_filters): This line applies the conv_block function to the inputs. The conv_block function consists of two sets of Conv2D -> BatchNormalization -> Activation layers. The output of this operation is stored in x.

2. p = MaxPool2D((2, 2))(x): This line applies a 2D max pooling operation to the output of the conv_block. Max pooling is a downsampling strategy that selects the maximum value from a region of the input. In this case, the region size is 2x2. The output of this operation is stored in p.

3. return x, p: The function returns two tensors. The first tensor x is the output of the conv_block (before pooling). This will be used later in the "upsampling" or "expansion" part of the network for the skip connections. The second tensor p is the output of the MaxPool2D layer. This will be the input to the next encoder_block in the network.

In the context of the U-Net architecture, the purpose of the encoder_block is to extract features from the input image at different scales and to reduce the spatial dimensions of the input for the next encoder_block.

In [7]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

## Decoder Block

1. x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs): This line applies a Conv2DTranspose operation to the inputs. Conv2DTranspose is often referred to as deconvolution in the context of CNNs. It performs an inverse convolution operation, which increases the spatial dimensions of the input. The number of filters, kernel size, and stride are specified as parameters.

2. x = Concatenate()([x, skip_features]): This line concatenates the upsampled tensor x and the corresponding tensor from the encoder path (skip_features). This is known as a skip connection, which helps to recover the spatial information lost during encoding.

3. x = conv_block(x, num_filters): This line applies the conv_block function to the concatenated tensor. The conv_block function consists of two sets of Conv2D -> BatchNormalization -> Activation layers.

4. return x: The function returns the output tensor, which will be the input to the next decoder_block in the network or the final output layer of the network.

In the context of the U-Net architecture, the purpose of the decoder_block is to upsample the feature map and concatenate it with the correspondingly downsampled feature map from the encoding path. This helps to recover the spatial information and details lost during the encoding process.

In [8]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

## Model Build Up
1. inputs = Input(input_shape): This line creates an input layer with the given shape.

2. The next four lines (s1, p1 = encoder_block(inputs, 64) and so on) define the encoder path of the U-Net. Each encoder_block consists of a convolutional block followed by a MaxPool2D layer for downsampling. The function returns two tensors: the output of the convolutional block (before pooling) and the output of the MaxPool2D layer.

3. b1 = conv_block(p4, 1024): This line defines the bottleneck of the U-Net, which is a convolutional block at the bottom of the U.

4. The next four lines (d1 = decoder_block(b1, s4, 512) and so on) define the decoder path of the U-Net. Each decoder_block consists of a Conv2DTranspose layer for upsampling, a concatenation of the upsampled tensor and the corresponding tensor from the encoder path (skip connection), and a convolutional block.

5. outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4): This line defines the output layer of the U-Net. It's a Conv2D layer with a sigmoid activation function, which is commonly used for binary classification problems.

6. model = Model(inputs, outputs, name="UNET"): This line creates a Keras Model instance with the given inputs and outputs.

 The purpose of the build_unet function is to construct the U-Net model with the specified input shape. The U-Net model is a type of convolutional neural network that is particularly effective for image segmentation tasks.

In [9]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [10]:
input_shape = (256, 256, 3)
model = build_unet(input_shape)
model.summary()



Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 64)         0         ['batch_normalization[0][

## Utility Functions for Metrics
1. dice_coef function: This function calculates the Dice coefficient, a metric for comparing the similarity of two samples. It's often used in image segmentation tasks. The Dice coefficient is defined as 2 * (the area of overlap) divided by (the total number of pixels in both images), ranging from 0 (no overlap) to 1 (perfect overlap). In this function, y_true and y_pred are the ground truth and the model's prediction, respectively. Both are flattened into 1D arrays before the calculation. The smooth term is added to the denominator to prevent division by zero.

2. dice_loss function: This function calculates the Dice loss, which is simply 1 minus the Dice coefficient. The Dice loss is a measure of dissimilarity between y_true and y_pred. A lower Dice loss indicates a better model performance.

In the context of a brain tumor segmentation task, these functions could be used to train a model to segment brain tumors from MRI images. The Dice coefficient would measure the model's performance, and the Dice loss would be minimized during training.

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Loading and Training on DATASET

In [12]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [13]:
""" Global parameters """
H = 256
W = 256

### Functions to build a temp path and load dataset

In [19]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))

    print("Images : ", len(images))
    print("Masks : ", len(masks))

    train_x, valid_x = train_test_split(images, test_size=0.2, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=0.2, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=0.25, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=0.25, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

### Reading of Image and Mask from the dataset

In [16]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))                   ## (h, w)
    x = x / 255.0                               ## (h, w)
    x = x.astype(np.float32)                    ## (h, w)
    x = np.expand_dims(x, axis=-1)              ## (h, w, 1)
    return x

In [17]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")

""" Hyperparameters """
batch_size = 16
lr = 1e-4
num_epochs = 5
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

In [21]:
""" Dataset """
dataset_path = "/content/drive/MyDrive/Major/dataset"
dataset_path_local = "D:\Major\Code\dataset"

path = dataset_path_local

(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(path)


Images :  3064
Masks :  3064


In [22]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

Train: 1838 - 1838
Valid: 613 - 613
Test : 613 - 613


In [23]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [24]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

Epoch 1/5


  1/115 [..............................] - ETA: 1:29:53 - loss: 0.9654 - dice_coef: 0.0346

KeyboardInterrupt: 